Addig annotations from SNPnexus output files

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from matplotlib import pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from bsmcalls import individuals
from bsmcalls import readVCF
from bsmcalls import SNPnexus
from bsmcalls import operations
from bsmcalls import preprocessing
import pandas as pd
import numpy as np
import re
import pickle
import os.path
import attila_utils
%matplotlib inline

## Annotation with SNPnexus
[SNPnexus](https://www.snp-nexus.org/v4/) is a recently updated web service that uses numerous databases to annotate human genomic variants (see [this article](https://academic.oup.com/nar/article/48/W1/W185/5851388)).  The short sections below present information regarding the way I used SNPnexus to annotate our somatic variant calls.  The following settings were used:

* Human Assembly: GRCh37/hg19
* filtered VCF files were uploaded
* all annotation categories were selected
* *TXT per annotation*

### Overlapping genes
A key annotation of *Overlapped and Nearest Genes*, stored in the `near_gens.txt` output file of SNPnexus.  Each variant is overlapped by zero, one or more genes. In the last case two or more genes overlap each other around the given variant and `near_gens.txt` lists the same variant in multiple rows corresponding to the multiple overlapping genes. Since my `calls` DataFrame must contain exactly one row for each variant I collapsed those multiple rows into one by listing the overlapping genes in a single colon (`:`) separated string.

[This article](https://www.nature.com/articles/s41598-019-49802-w) presents a study of overlapping genes and shows that sometimes more than 5 genes may overlap at a locus.  This means that for each variant I should come up with a---possibly empty---set of genes that overlap that variant.

### Selecting sets of annotations

See `notebook/2020-09-07-annotations/examine-annot-files.csv`, which lists all the annotation files and whether they were incuded in further analysis based on two criteria

1. redundance: several sets annotations provide very similar information therefore only one of those were included
2. relevance: cancer related annotations were excluded

## Implementation

See `SNPnexus.do_annot` for details

In [2]:
calls = individuals.get_datasets()
fpath = '/home/attila/projects/bsm/results/2020-09-07-annotations/annot'
picklepath = fpath + '.p'
csvpath = fpath + '.csv'
if not os.path.exists(picklepath):
    annot_raw = SNPnexus.do_annot(calls=calls)
else:
    print('loading raw annotations DataFrame from', picklepath)
    with open(picklepath, 'rb') as f:
        annot_raw = pickle.load(f)
print('writing raw annotations DataFrame to CSV', csvpath)
annot_raw.to_csv(csvpath)

loading raw annotations DataFrame from /home/attila/projects/bsm/results/2020-09-07-annotations/annot.p
writing raw annotations DataFrame to CSV /home/attila/projects/bsm/results/2020-09-07-annotations/annot.csv


In [3]:
fpath = '/home/attila/projects/bsm/results/2020-09-07-annotations/annotated-calls'
# write files without filtering
data = SNPnexus.merge_clean(calls, annot_raw, dofilter=False)
data.to_csv(fpath + '-full.csv')
pickle.dump(data, open(fpath + '-full.p', 'wb'))
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11824 entries, ('AN02255', 'frontal cortex', '1', 4798634, 'T/G') to ('UMB914', 'frontal cortex', 'X', 130133640, 'C/T')
Columns: 133 entries, REF to remm_ReMM Score
dtypes: Int64(23), category(47), float64(50), int64(2), object(11)
memory usage: 9.7+ MB


Note that these data contain superfluous observations:
* NeuN_mn or muscle tissue from the Chess dataset
* low quality calls for the Walsh dataset

We remove these with `dofilter=True`

In [4]:
# repeat with filtering
data = SNPnexus.merge_clean(calls, annot_raw, dofilter=True)
data.to_csv(fpath + '.csv')
pickle.dump(data, open(fpath + '.p', 'wb'))
#del annot_raw
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6426 entries, ('CMC_MSSM_027', '1', 11973569, 'C/T') to ('UMB914', 'X', 130133640, 'C/T')
Columns: 133 entries, REF to remm_ReMM Score
dtypes: Int64(23), category(47), float64(50), int64(2), object(11)
memory usage: 5.4+ MB


## How to load data?

See implementation: `SNPnexus.load_data()`

In [5]:
fpath = '/home/attila/projects/bsm/results/2020-09-07-annotations/annotated-calls.p'
with open(fpath, 'rb') as f:
    data = pickle.load(f)

## Some checks
Let's look at the consistency between the `ensembl_.*` and the `near_gens_Overlapped Gene` columns!  There are more calls annotated (i.e not `NaN`) by `ensembl_.*` than by `near_gens_Overlapped Gene`:

In [6]:
ensembl_pred_fun = operations.anyquery('ensembl_Predicted Function', data).iloc[:, 0]
near_gens_overlap = operations.anyquery('near_gens_Overlapped Gene', data).iloc[:, 0]
print(sum(ensembl_pred_fun & ~ near_gens_overlap), 'non overlapping genes in with some predicted function in ensembl')
print(sum(~ ensembl_pred_fun & near_gens_overlap), 'overlapping genes in without any predicted function in ensembl')

263 non overlapping genes in with some predicted function in ensembl
0 overlapping genes in without any predicted function in ensembl


But the only inconsistency between the `ensembl_.*` and the `near_gens_Overlapped Gene` columns is at where `ensembl_Predicted Function` is either `3downstream` or `5upstream`

In [7]:
s = data.loc[(ensembl_pred_fun & ~ near_gens_overlap), 'ensembl_Predicted Function']
S = set()
s.dropna().apply(lambda x: S.update(x))
S

{'3downstream', '5upstream'}

The conclusion from this bit of analysis is that the `ensembl_Symbol` column has the same information as `near_gens_Overlapped Gene` and the thus `ensembl_Gene` can be used to query with stable ensembl IDs instead of unstable symbols (gene names).

## Andy's questions

* Roadmap epigenome annotation of SNPnexus: what does it exactly mean?
* mutation types (A/C, ...): how does the mutational spectrum in the outlier sample compare to that in other samples?
* brain expressed genes combined with chromatin state DLPFC
* control callsets from other BSMN groups

In [8]:
%connect_info

{
  "shell_port": 44005,
  "iopub_port": 41107,
  "stdin_port": 59801,
  "control_port": 52345,
  "hb_port": 43023,
  "ip": "127.0.0.1",
  "key": "07abedbc-7752c57903b8267c720ba677",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-11bbdcbc-e740-4e7a-a832-cd580a9dd664.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
